# Quantum-Inspired Acromyrmex Evolutionary Algorithm (QIAEA) in Graph Clustering

The notebook contains first experimental algorithm implementation for graph clustering problem via QIAEA.

The graph for algorithm is built on the dataset "Relato Business Graph Database", that can be downloaded by <a href="https://www.kaggle.com/datasets/thedevastator/relato-business-network-graph-373663-domain-conn">link</a>.

## Data Download and Clear

In [12]:
import pandas as pd
import numpy as np

In [13]:
links = pd.read_csv("dataset/links.csv")

C:\Users\fatkh\AppData\Local\Temp\ipykernel_16500\2811395870.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv("dataset/links.csv")


In [14]:
links = links.drop(['index', '_id', 'update_time', 'home_domain', 'link_domain', 'username', 'name'], axis=1)
links = links.dropna()
links = links[links['type'] == 'partnership'].drop(['type'], axis=1)
links = links.drop_duplicates()
links.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112675 entries, 0 to 360547
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   home_name  112675 non-null  object
 1   link_name  112675 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB


In [15]:
companies = np.unique_values(
    np.concatenate(
        (links['home_name'].unique(), links['link_name'].unique())
        ).astype('str')
    )
companies = np.sort(companies)
len(companies)

43065

In [16]:
map_companies = dict()
for idx, company in enumerate(companies):
    map_companies[company] = idx

In [17]:
edges = []
for link in links.values:
    edges.append([map_companies[link[0]], map_companies[link[1]]])
    
edges = np.array(edges)
len(edges)

112675

# Class Declarations

- **Graph** - object that contain all the information about the graph: vertices, edges, degrees of vertices, modularity calculation;
- **Partition** - object that represents clusterization of a graph. The Partition consists of quantum and collapsed (determined) chromosomes.

In [18]:
class Graph:
    def __init__(self, vertices: np.ndarray, edges: np.ndarray):
        self.number_of_vertices = len(vertices)
        self.number_of_edges = len(edges)
        self.vertices = vertices
        self.edges = edges
        assert edges.shape[1] == 2
        self.degrees = np.zeros((self.number_of_vertices,))

        for edge in edges:
            self.degrees[edge[0]] += 1
            self.degrees[edge[1]] += 1

    def modularity(self, labels: np.ndarray, k_max: int):
        degree_per_cluster = np.bincount(labels, self.degrees, minlength=k_max)

        from_clusters = labels[edges[:, 0]]
        to_clusters = labels[edges[:, 1]]

        mask = (from_clusters == to_clusters)

        edges_per_cluster = np.bincount(from_clusters[mask], minlength=k_max).astype('int32')

        modularity = np.sum(edges_per_cluster / self.number_of_edges) - np.sum((degree_per_cluster / (2 * self.number_of_edges)) ** 2)

        return modularity

In [19]:
class Partition:
    def __init__(self, number_of_vertices: np.ndarray, k_max: int):
        assert k_max < 256
        self.number_of_vertices = number_of_vertices
        self.k_max = k_max
        self.fitness = -0.5
        self.gene_len = 1

        while 2**(self.gene_len) < k_max:
            self.gene_len += 1

        self.chromosome = np.random.rand(number_of_vertices, self.gene_len, 2)
        self.chromosome[:, :, 1] = np.sqrt(1 - self.chromosome[:, :, 0]**2)
        self.collapsed_chromosome = None

    def __copy__(self):
        obj = Partition(self.number_of_vertices, self.k_max)
        obj.fitness = self.fitness
        obj.gene_len = self.gene_len
        obj.chromosome = np.copy(self.chromosome)
        obj.collapsed_chromosome = np.copy(self.collapsed_chromosome)
        return obj

    def collapse(self):
        labels = np.zeros((self.number_of_vertices,), dtype='int32')
        measure = np.random.rand(self.chromosome.shape[0], self.chromosome.shape[1])
        measured_chromosome = self.chromosome[:, :, 1] > measure
        labels = np.packbits(measured_chromosome, axis=1, bitorder='little').flatten()
        labels = labels % self.k_max
        self.collapsed_chromosome = labels

    def calculate_fitness(self, graph: Graph):
        self.fitness = graph.modularity(self.collapsed_chromosome, self.k_max)
        self.fitness = (self.fitness + 0.5) * 2 / 3

    def mutation(self, number_of_mutations: int = 1):
        assert number_of_mutations >= 1

        new_partition = Partition(self.number_of_vertices, self.k_max)
        new_partition.chromosome = np.copy(self.chromosome)

        for _ in range(number_of_mutations):
            gene, location = np.random.randint(0, self.number_of_vertices), np.random.randint(0, self.gene_len)
            alpha, beta = new_partition.chromosome[gene][location][0], new_partition.chromosome[gene][location][1]
            new_partition.chromosome[gene][location][0], new_partition.chromosome[gene][location][1] = beta, alpha

        return new_partition


def crossover(partition1: Partition, partition2: Partition) -> Partition:
    child = Partition(partition1.number_of_vertices, partition1.k_max)

    adamart_operator = np.array([
        [1, 1],
        [1, -1]
    ]) / np.sqrt(2)

    xgate_operator = np.array([
        [0, 1],
        [1, 0]
    ])

    mask_adamart = (partition1.collapsed_chromosome == partition2.collapsed_chromosome)
    mask_xgate = (partition1.collapsed_chromosome != partition2.collapsed_chromosome)
    child.chromosome[mask_adamart] = child.chromosome[mask_adamart] @ adamart_operator.T
    child.chromosome[mask_xgate] = child.chromosome[mask_xgate] @ xgate_operator.T
    return child



## Algorithm Implementation
The QIAEA is applied for graph clusterization problem to a specific dataset.

Key details:
- Each vertex in partitions has length B, where <math>$$2^{B-1} < \#\_of\_clusters < 2^B$$</math>
- In case number of clusters less than 2**B, the first clusters are defined by two numbers
- Crossover operation is fully implemented as described in the <a href="https://www.nature.com/articles/s41598-019-48409-5">paper</a>
- Mutation operation replace alpha and beta of random allels, number of mutations is a parameter.
- Measurement operation is fully implemented as descrived in the <a href="https://www.nature.com/articles/s41598-019-48409-5">paper</a>. This operation is necessary for calculating fitnees, as soon as fitness is defined only on determined state of an organism
- Modularity is used as a graph clusterization metric. Modularity is in the range [-0.5, 1].
- Fitness value is based on the modularity and is calculated by the formula: <math>$$fitness = {{2 * (modularity + 0.5)}\over{3}}$$</math>, so the fitness value is in range [0, 1]

Notice that most of the graphs do not have clustering with high modularity. The high modularity cannot be achieved in such graphs due to its structure, where there are no actual groups of nodes.

In [23]:
population_size = 500
number_of_males = population_size // 2
max_number_of_clusters = 32
number_of_epochs = 15
graph = Graph(companies, edges)

In [26]:
population = [Partition(len(companies), max_number_of_clusters) for _ in range(population_size)]

In [27]:
best_partition = population[0].__copy__()
for partition in population:
    partition.collapse()
    partition.calculate_fitness(graph)
population.sort(key=lambda x: -x.fitness)

for epoch in range(1, number_of_epochs+1):
    print("----------------------------------")
    # Crossover operations
    for i in range(1, number_of_males+1):
        population.append(crossover(population[0], population[i]))
    # Mutation operations
    for i in range(1, population_size):
        population.append(population[i].mutation())
    # Measure operations
    for i in range(population_size, len(population)):
        population[i].collapse()
    # Fitness calculation
    for i in range(population_size, len(population)):
        population[i].calculate_fitness(graph)
    population.sort(key=lambda x: -x.fitness)

    population = population[:population_size]

    if population[0].fitness > best_partition.fitness:
        best_partition = population[0].__copy__()

    print(f"Epoch {epoch}, best fitness: {best_partition.fitness}")


----------------------------------
Epoch 1, best fitness: 0.3339381308504367
----------------------------------
Epoch 2, best fitness: 0.33401971277346565
----------------------------------
Epoch 3, best fitness: 0.33401971277346565
----------------------------------
Epoch 4, best fitness: 0.33437183333071596
----------------------------------
Epoch 5, best fitness: 0.33437183333071596
----------------------------------
Epoch 6, best fitness: 0.33437183333071596
----------------------------------
Epoch 7, best fitness: 0.33437183333071596
----------------------------------
Epoch 8, best fitness: 0.3347936620642229
----------------------------------
Epoch 9, best fitness: 0.3347936620642229
----------------------------------
Epoch 10, best fitness: 0.3347936620642229
----------------------------------
Epoch 11, best fitness: 0.3347936620642229
----------------------------------
Epoch 12, best fitness: 0.3347936620642229
----------------------------------
Epoch 13, best fitness: 0.334793

In [28]:
clusters = [[] for _ in range(max_number_of_clusters)]

for vertex, cluster in enumerate(best_partition.collapsed_chromosome):
    clusters[cluster].append(companies[vertex])

print("Best partition clustering:")
for idx, cluster in enumerate(clusters, 1):
    print(f"Number of vertices of cluster {idx}: {len(cluster)}")

Best partition clustering:
Number of vertices of cluster 1: 26
Number of vertices of cluster 2: 55
Number of vertices of cluster 3: 85
Number of vertices of cluster 4: 249
Number of vertices of cluster 5: 82
Number of vertices of cluster 6: 266
Number of vertices of cluster 7: 253
Number of vertices of cluster 8: 943
Number of vertices of cluster 9: 77
Number of vertices of cluster 10: 266
Number of vertices of cluster 11: 262
Number of vertices of cluster 12: 949
Number of vertices of cluster 13: 277
Number of vertices of cluster 14: 1007
Number of vertices of cluster 15: 978
Number of vertices of cluster 16: 3449
Number of vertices of cluster 17: 70
Number of vertices of cluster 18: 300
Number of vertices of cluster 19: 259
Number of vertices of cluster 20: 964
Number of vertices of cluster 21: 265
Number of vertices of cluster 22: 971
Number of vertices of cluster 23: 948
Number of vertices of cluster 24: 3435
Number of vertices of cluster 25: 272
Number of vertices of cluster 26: 9